In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import copy

2023-09-13 21:27:44.209986: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 21:27:44.678295: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
batch_size = 1024
learning_rate = 0.001

In [3]:
@tf.keras.saving.register_keras_serializable()
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(units=128, activation=tf.nn.leaky_relu)
        self.dense2 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
        self.dense3 = tf.keras.layers.Dense(units=128, activation=tf.nn.leaky_relu)
        self.dense4 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
        self.dense5 = tf.keras.layers.Dense(units=8)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        output = self.dense5(x)
        return output

In [4]:
class ParaServer:
    def __init__(self):
        self.model = MLP()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    def upload(self, grads):
        self.optimizer.apply_gradients(grads_and_vars=zip(grads, self.model.variables))
        return self.model
    def download(self):
        return self.model
    def initModel(self, x):
        self.model(x)

In [5]:
def valiAll():
    m = ps.download()
    model = copy.deepcopy(m)
    y_v_p = model(X_v)
    va_mse = tf.reduce_mean(tf.square(y_v_p - y_v))
    va_rmse = tf.sqrt(va_mse)
    va_mae = tf.reduce_mean(tf.abs(y_v_p - y_v))
    va_r2 = 1 - tf.reduce_sum(tf.square(y_v_p - y_v)) / tf.reduce_sum(tf.square(y_v - tf.reduce_mean(y_v)))
    print("mse:{} rmse:{} mae:{} r2:{}".format(va_mse, va_rmse, va_mae, va_r2))
    r2sv.append(va_r2.numpy())

In [12]:
class Node:
    def __init__(self, dsName,freq):
        self.model = MLP()
        self.freq = freq
        dataset = pd.read_csv(dsName, encoding='utf-8').sample(frac=1).reset_index(drop=True)
        self.X = dataset.loc[:,'freq':'L4'].to_numpy(dtype = np.float32)
        self.y = dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)
        self.dataset_train = tf.data.Dataset.from_tensor_slices((self.X, self.y))
        self.dataset_train = self.dataset_train.shuffle(buffer_size=23000)
        self.dataset_train = self.dataset_train.batch(batch_size)
        self.dataset_train = self.dataset_train.prefetch(tf.data.experimental.AUTOTUNE)
    def train(self, num_epochs):
        m = ps.download()
        self.model = copy.deepcopy(m)
        for epoch_index in range(num_epochs):
            for X, y in self.dataset_train:
                with tf.GradientTape() as tape:
                    y_pred = self.model(X)
                    tr_mse = tf.reduce_mean(tf.square(y_pred - y))
                tr_rmse = tf.sqrt(tr_mse)
                tr_mae = tf.reduce_mean(tf.abs(y_pred - y))
                tr_r2 = 1 - tf.reduce_sum(tf.square(y_pred - y)) / tf.reduce_sum(tf.square(y - tf.reduce_mean(y)))
                grads = tape.gradient(tr_mse, self.model.variables)
                m = ps.upload(grads)
                self.model = copy.deepcopy(m)
                # if epoch_index in np.arange(0, num_epochs, 25).tolist() or epoch_index == num_epochs - 1:
            if True:
                print("node:{} epoch:{}".format(self.freq, epoch_index))
                print("train mse:{} rmse:{} mae:{} r2:{}".format(tr_mse, tr_rmse, tr_mae, tr_r2))
                r2s.append(tr_r2.numpy())
                valiAll()

In [13]:
r2s = []
r2sv = []

In [14]:
test_dataset = pd.read_csv("Test.csv", encoding='utf-8').sample(frac=1).reset_index(drop=True)
X_v = test_dataset.loc[:,'freq':'L4'].to_numpy(dtype = np.float32)
y_v = test_dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)

In [15]:
ps = ParaServer()
ps.initModel(X_v)

In [16]:
nodeList = [Node('./24Train.csv', 2.4), Node('./25Train.csv', 2.5), Node('./26Train.csv', 2.6)]

In [17]:
nodeList[0].train(150)
nodeList[1].train(150)
nodeList[2].train(150)
nodeList[0].train(150)
nodeList[1].train(150)
nodeList[2].train(150)

node:2.4 epoch:0
train mse:0.1409413367509842 rmse:0.37542152404785156 mae:0.3049042820930481 r2:-0.16492414474487305
mse:0.11417369544506073 rmse:0.3378959894180298 mae:0.2753213346004486 r2:0.05343145132064819
node:2.4 epoch:1
train mse:0.09114372730255127 rmse:0.301900178194046 mae:0.24838295578956604 r2:0.24938338994979858
mse:0.0951123759150505 rmse:0.3084029257297516 mae:0.2533417344093323 r2:0.21146124601364136
node:2.4 epoch:2
train mse:0.08542017638683319 rmse:0.2922672927379608 mae:0.23830635845661163 r2:0.29189765453338623
mse:0.08993029594421387 rmse:0.29988378286361694 mae:0.2449856698513031 r2:0.2544237971305847
node:2.4 epoch:3
train mse:0.08246717602014542 rmse:0.28717097640037537 mae:0.23345564305782318 r2:0.3194320797920227
mse:0.08800553530454636 rmse:0.2966572642326355 mae:0.24071508646011353 r2:0.27038121223449707
node:2.4 epoch:4
train mse:0.08057672530412674 rmse:0.28386038541793823 mae:0.23033808171749115 r2:0.33324772119522095
mse:0.0860355868935585 rmse:0.2933

In [21]:
for i in r2sv:
    print(i)

0.05343145
0.21146125
0.2544238
0.2703812
0.2867133
0.30649197
0.31547016
0.3224408
0.3291536
0.3452314
0.34077
0.3562141
0.354967
0.35245204
0.32595384
0.40231603
0.40158945
0.40962702
0.42272305
0.4230671
0.4107265
0.41005546
0.436269
0.44843858
0.43839198
0.43388087
0.45969415
0.45770818
0.4541483
0.4664585
0.47532052
0.46199465
0.47981328
0.47990716
0.49459165
0.4690683
0.45069534
0.48096782
0.49847054
0.46773773
0.4992084
0.47926748
0.49972522
0.48676974
0.4963382
0.48902565
0.5175091
0.50038993
0.46291137
0.5333949
0.53417766
0.48639023
0.55056655
0.5456645
0.53572404
0.5499059
0.5739113
0.5726086
0.47727573
0.5278763
0.49325925
0.58120906
0.47513604
0.5654805
0.5336244
0.4959963
0.5241405
0.5895729
0.45611674
0.37396538
0.50237
0.46507394
0.513464
0.49617326
0.5751781
0.5460856
0.4678169
0.5107386
0.5489664
0.61125076
0.45940083
0.43182415
0.49852914
0.52809775
0.47194016
0.5567453
0.496157
0.3778242
0.471923
0.41169584
0.5299705
0.43949437
0.41430855
0.38863832
0.47739798
0.538